In [10]:
## for senet two images

import caffe
import numpy as np

batch_size = 8

#weights_file = 'caffe_model/senet50_ft_caffe/senet50_ft.caffemodel'
#model_file = 'caffe_model/senet50_ft_caffe/senet50_ft_feature.prototxt'

#weights_file = 'caffe_model/senet50_scratch_caffe/senet50_scratch.caffemodel'
#model_file = 'caffe_model/senet50_scratch_caffe/senet50_scratch_feature.prototxt'

#weights_file = 'caffe_model/resnet50_ft_caffe/resnet50_ft.caffemodel'
#model_file = 'caffe_model/resnet50_ft_caffe/resnet50_ft_feature.prototxt'

weights_file = 'caffe_model/resnet50_scratch_caffe/resnet50_scratch.caffemodel'
model_file = 'caffe_model/resnet50_scratch_caffe/resnet50_scratch_feature.prototxt'


caffe.set_mode_gpu()
net = caffe.Net(model_file, weights_file, caffe.TEST)

# create transformer for the input called 'data'
mu = np.array([91.4953, 103.8827, 131.0912])

transformer = caffe.io.Transformer({'data': net.blobs['data'].data.shape})

transformer.set_transpose('data', (2,0,1))  # move image channels to outermost dimension
transformer.set_mean('data', mu)            # subtract the dataset-mean value in each channel
transformer.set_raw_scale('data', 255)      # rescale from [0, 1] to [0, 255]
transformer.set_channel_swap('data', (2,1,0))  # swap channels from RGB to BGR

net.blobs['data'].reshape(batch_size,        # batch size
                          3,         # 3-channel (BGR) images
                          224, 224)

In [11]:
import time

image1_path = "image/8.jpg_chip.png"
image2_path = "image/125.jpg_chip.png"

#image1_path = "image/5.jpg"
#image2_path = "image/124.jpg"


def cos_similar(a, b):
    A = np.array(a)
    B = np.array(b)
    
    num = np.sum(A*B)
    denom = np.linalg.norm(A) * np.linalg.norm(B)  
    
    cos = num / denom
    sim = 0.5 + 0.5 * cos
    
    return sim

## load and fill image
#image = caffe.io.load_image(image1_path)
#transformed_image = transformer.preprocess('data', image)
#net.blobs['data'].data[0] = transformed_image

#image = caffe.io.load_image(image2_path)
#transformed_image = transformer.preprocess('data', image)
#net.blobs['data'].data[1] = transformed_image

imgs_1 = np.zeros((batch_size, 3, 224, 224))
imgs_2 = np.zeros((batch_size, 3, 224, 224))


img1 = caffe.io.load_image(image1_path)
x1 = transformer.preprocess('data', img1)
imgs_1[0] = x1
imgs_1[1] = x1

img2 = caffe.io.load_image(image2_path)
x2 = transformer.preprocess('data', img2)
imgs_2[0] = x2
imgs_2[1] = x2
    
net.blobs['data'].data[...] = imgs_1
output1 = net.forward()

start = time.time()
net.forward()
end = time.time()

print ("{} ms".format((end-start)*1000))
preds_1 = output1['flatten'].copy()

net.blobs['data'].data[...] = imgs_2
output2 = net.forward()
%timeit net.forward()
preds_2 = output2['flatten'].copy()

print (preds_1)
print (preds_2)

ogi = 1/(1+np.linalg.norm(preds_1[0]-preds_2[0]))

print ("cos Similar is {}, ogi is {}".format(cos_similar(preds_1[0], preds_2[0]), ogi))


140.21396637 ms
10 loops, best of 3: 111 ms per loop
[[0.84799814 0.         0.9964056  ... 0.03545221 0.6139068  0.33769438]
 [0.84799814 0.         0.9964056  ... 0.03545221 0.6139068  0.33769438]
 [0.         0.         0.         ... 0.02876952 0.         0.        ]
 ...
 [0.         0.         0.         ... 0.02876952 0.         0.        ]
 [0.         0.         0.         ... 0.02876952 0.         0.        ]
 [0.         0.         0.         ... 0.02876952 0.         0.        ]]
[[0.02687134 0.         1.5585577  ... 0.01613929 0.8262487  0.36209238]
 [0.02687134 0.         1.5585577  ... 0.01613929 0.8262487  0.36209238]
 [0.         0.         0.         ... 0.02876952 0.         0.        ]
 ...
 [0.         0.         0.         ... 0.02876952 0.         0.        ]
 [0.         0.         0.         ... 0.02876952 0.         0.        ]
 [0.         0.         0.         ... 0.02876952 0.         0.        ]]
cos Similar is 0.987234503031, ogi is 0.0692367337078
